In [3]:
#from PIL import Image
from pathlib import Path
from os import path
import numpy as np
import cv2
#import imutils
from time import sleep

IMG_DIR = Path("handwriting")
CROPPED_DIR = Path("cropped")
DESKEWED_DIR = Path("deskewed")


counter = 1
for picture in CROPPED_DIR.iterdir():
        
    if picture.suffix in [".jpg", ".jpeg", ".png"] and not picture.name.startswith("."):
        file_name = str(counter)+".jpg"
        file_path = path.join(Path(CROPPED_DIR, file_name))
        save_name = str(file_path)

#Crop
        # reduce the area
##        left = 50
##        top = 50
##        right = 1250
##        bottom = 750

        img = cv2.imread(str(picture.resolve()), 1) #1 is grayscale enum flag  
##        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
##        img = img[top:bottom, left:right]
##        cv2.imshow("test", img)
##        k = cv2.waitKey(0)
##        cv2.imwrite(f"{CROPPED_DIR}/{counter}.jpg", img)

#Threshold
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.bitwise_not(img) # for the threshold function only

        # threshold the image, setting all foreground pixels to 255 and all background pixels to 0
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#Deskew
        # grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
        # compute a rotated bounding box that contains all coordinates
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]

        # the `cv2.minAreaRect` function returns values in the range [-90, 0); as the rectangle rotates clockwise 
        # the returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle
        if angle < -45:
            angle = -(90 + angle)

        # otherwise, just take the inverse of the angle to make it positive
        else:
            angle = -angle

        # rotate the image to deskew it
        (h, w) = img.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        deskewd = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        cv2.imwrite(f"{DESKEWED_DIR}/{counter}.jpg", deskewd)

        counter += 1   
cv2.destroyAllWindows()